In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.decomposition as sk
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression

import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Load data file

In [ ]:
path2017 ='D:\Jupyter_File\Hackaton_Friday\kwb-2017.xls'
path2016 ='D:\Jupyter_File\Hackaton_Friday\kwb-2016.xls'
path2015 ='D:\Jupyter_File\Hackaton_Friday\kwb-2015.xls'
data = 'D:\Jupyter_File\Hackaton_Friday\Data.xls'

df_ori_2017 = pd.read_excel(path2017,index_col=None, header=0, convert_float= True,decimal =',')
df_ori_2016 = pd.read_excel(path2016,index_col=None, header=0, convert_float= True,decimal =',')
df_ori_2015 = pd.read_excel(path2015,index_col=None, header=0, convert_float= True,decimal =',')
data_ori = pd.read_excel(data,index_col=None, header=0, convert_float= True)

print('KWB 2017 Data consists :',len(df_ori_2017))
print('KWB 2016 Data consists :',len(df_ori_2016))
print('KWB 2015 Data consists :',len(df_ori_2015))
print('Translated Data consists :',len(data_ori))

In [ ]:
data_ori.head(2)

## Define which data categories

In [ ]:
# 2017 data
general_info_column_name = ['regio','gm_naam','recs','gwb_code']
bevolking_column_name =  ['a_inw', 'a_man', 'a_vrouw', 'a_00_14', 'a_15_24', 
                            'a_25_44', 'a_45_64', 'a_65_oo', 
                          'a_ongeh', 'a_gehuwd', 'a_gesch', 'a_verwed', 'a_w_all', 'a_nw_all', 
                            'a_marok', 
                          'a_antaru', 
                        'a_suri', 'a_tur','a_ov_nw', 
                          #'a_geb', 'p_geb', 'a_ste', 'p_ste', 
                         'a_hh', 'a_1p_hh', 'a_hh_z_k', 'a_hh_m_k', 'g_hhgro', 'bev_dich'
                            ]

stedelijkheid_column_name = ['ste_mvs','ste_oad']

wonen_column_name = ['a_woning', 'g_woz', 
                    'p_1gezw', 'p_mgezw' ,
                    'p_bewndw','p_leegsw','p_koopw',
                    'p_huurw','p_wcorpw',
                    'p_ov_hw','p_e_o_w','p_bjj2k','p_bjo2k'
                    ]

# 2016 data

bedrijfsvestigingen_column_name = ['a_bedv','a_bed_a','a_bed_bf','a_bed_gi','a_bed_hj','a_bed_kl','a_bed_mn','a_bed_ru']

voorzieningen_column_name = ['g_afs_hp','g_afs_gs','g_afs_kv','g_afs_sc','g_3km_sc']

# 2015 data
inkomen_column_name = ['a_inkont', 'g_ink_po' ,
                       'g_ink_pi', 'p_ink_li' ,'p_ink_hi', 'p_n_act', 'p_hh_li', 'p_hh_hi', 
                       'p_hh_lkk' ,'p_hh_osm']


# Complete data with other data frame

In [ ]:
combined_all_data_date_2017 = general_info_column_name + bevolking_column_name + wonen_column_name + stedelijkheid_column_name 
combined_all_data_date_2016 = general_info_column_name + bedrijfsvestigingen_column_name + voorzieningen_column_name
combined_all_data_date_2015 = general_info_column_name + inkomen_column_name

df_ready2017 = df_ori_2017[combined_all_data_date_2017]
df_ready2016 = df_ori_2016[combined_all_data_date_2016]
df_ready2015 = df_ori_2015[combined_all_data_date_2015]

df_ready_all = pd.merge(df_ready2017, df_ready2016, on=[ 'regio', 'gm_naam', 'recs', 'gwb_code'])
df_ready_all = pd.merge(df_ready_all, df_ready2015, on=[ 'regio', 'gm_naam', 'recs', 'gwb_code'])

In [ ]:
df_ready_all.head(2)

# Separate data based on their regional level

In [ ]:

df_netherlands_gementee = df_ready_all[df_ready_all['recs'].str.contains("Gemeente")].set_index('gm_naam')

df_den_haag = df_ready_all[df_ready_all['gm_naam'].str.contains("'s-Gravenhage")]

df_den_haag_wijk = df_den_haag[df_den_haag['recs'].str.contains("Wijk")].set_index('regio')

df_den_haag_buurt = df_den_haag[df_den_haag['recs'].str.contains("Buurt")].set_index('regio')

In [ ]:
df_den_haag_wijk.head(5)

In [ ]:
data_ori.head(2)

# Select data that contains number (not general name) only

In [ ]:
df_den_haag_wijk_value_column = df_den_haag_wijk.loc[:,'a_25_44':'g_ink_pi'].columns

# Convert data that in "object" format into numeric

In [ ]:
district = df_den_haag_wijk[df_den_haag_wijk_value_column]

In [ ]:
# check how many strings (object)
district.dtypes.value_counts()

In [ ]:
data_ori.dtypes.value_counts()

In [ ]:
# convert strings (object) into numeric
district = district.convert_objects(convert_numeric=True)

In [ ]:
# check how many strings (object) left
district.dtypes.value_counts()

In [ ]:
# fill NaN values with 0
district = district.fillna(0)
data_ori = data_ori.fillna(0)

In [ ]:
#district[district.dtypes[(district.dtypes=="float64")|(district.dtypes=="int64")].index.values].hist(figsize=[11,11])

# Normalize the dataframe

In [ ]:
plt.figure(figsize = (16,8))

sns.heatmap(district, annot = True)

In [ ]:
plt.figure(figsize = (16,8))

sns.heatmap(data_ori, annot = True)

In [ ]:
# MinMaxScaler method
district_normalize = preprocessing.MinMaxScaler().fit_transform(district)
# MaxAbsScaler method
district_normalize2 = preprocessing.MaxAbsScaler().fit_transform(district)

data_ori_normalize2 = preprocessing.MaxAbsScaler().fit_transform(data_ori)

# normalize method
district_normalize3 = preprocessing.normalize(district, norm='l2')
# StandardScaler method
district_normalize4 = preprocessing.StandardScaler().fit_transform(district)



In [ ]:
#sns.heatmap(neighboorhood_normalize)
#sns.heatmap(neighboorhood_normalize3)
#sns.heatmap(neighboorhood_normalize4)

In [ ]:
district.loc[:,:] = district_normalize2
data_ori.loc[:,:] = data_ori_normalize2

In [ ]:
plt.figure(figsize = (16,8))

sns.heatmap(district, annot = True)

In [ ]:
plt.figure(figsize = (16,8))

sns.heatmap(data_ori, annot = True)

# Start to do NMF with the Normalized data

In [ ]:
from sklearn.decomposition import NMF

samples = data_ori.as_matrix()

# Loading KMeans model
# The number of clusters
N_com = 7

model1 = NMF(n_components = N_com)

# Clustering the data
model1.fit(samples)

# Getting the labes of the clusters
#labels = model1.predict(samples)

# Trying to find an optimal number of clusters using inertia
ks = range(1, 20)
inertias = []

for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters = k)
    
    # Fit model to samples
    model.fit(samples)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
model = NMF(n_components = N_com)
W = model.fit_transform(district)
H = model.components_

In [ ]:
district_name = district.index
variable_name = district.columns

NMF_W = pd.DataFrame(index=district_name, columns= list(range(0, N_com)))
NMF_H = pd.DataFrame(index= list(range(0, N_com)) , columns= variable_name)

In [ ]:
NMF_W.loc[:,:] = W
NMF_H.loc[:,:] = H

In [ ]:
plt.figure(figsize = (16,8))
sns.heatmap(NMF_H, annot = True)

In [ ]:
plt.figure(figsize = (16,8))
sns.clustermap(NMF_W.T, annot = True)

In [ ]:
plt.figure(figsize = (16,8))
sns.heatmap(NMF_W, annot = True)

In [ ]:
plt.figure(figsize = (16,8))
sns.clustermap(NMF_H, annot = True)

In [ ]:
model = NMF(n_components = N_com)
W = model.fit_transform(data_ori)
H = model.components_

In [ ]:
data_ori_district_name = data_ori.index
data_ori_variable_name = data_ori.columns


NMF_W = pd.DataFrame(index=data_ori_district_name, columns= list(range(0, N_com)))
NMF_H = pd.DataFrame(index= list(range(0, N_com)) , columns= data_ori_variable_name)

NMF_W.loc[:,:] = W
NMF_H.loc[:,:] = H

In [ ]:
plt.figure(figsize = (16,8))
sns.heatmap(NMF_H.T, annot = True)

In [ ]:
(NMF_H)

In [ ]:
plt.figure(figsize = (16,8))
sns.heatmap(NMF_W, annot = True)